<a href="https://colab.research.google.com/github/YunusKaratepe/duygu-tanima/blob/main/application_yunus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Must

In [41]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [111]:
# navigate your folder that u wanna save spectrograms.
%cd /content/drive/MyDrive/colab/final_project/app

/content/drive/MyDrive/colab/final_project/app


In [69]:
import os
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from keras.models import Model
from keras.models import load_model

import cv2

In [102]:
emotions_dict = {
        0: 'Neutral',
        1: 'Calm',
        2: 'Happy',
        3: 'Sad',
        4: 'Angry',
        5: 'Fearful',
        6: 'Disgusted',
        7: 'Surprised'
    }

In [65]:
def plot_spectrogram(Y, sr, hop_length, y_axis="log"):
    plt.figure(figsize=(25, 10))
    plt.axis('off')
    librosa.display.specshow(Y,
                             sr=sr,
                             hop_length=hop_length,
                             y_axis=y_axis)

    plt.savefig("./spectrogram.png", bbox_inches='tight', pad_inches=0)
    plt.close()

In [113]:
def create_spectrogram(order: int, spec_type: str):
    voice, sr = librosa.load(audio_file)
    after_stft = librosa.stft(voice, n_fft=FRAME_SIZE, hop_length=HOP_LENGTH)
    abs_stft_result = np.abs(after_stft) ** 2
    log_stft_result = librosa.power_to_db(abs_stft_result)
    if order != 0:
        log_stft_delta = librosa.feature.delta(log_stft_result, order)
        plot_spectrogram(log_stft_delta, sr, HOP_LENGTH, y_axis=spec_type)
        return
    plot_spectrogram(log_stft_result, sr, HOP_LENGTH, y_axis=spec_type)

In [108]:
def predict_emotion(cnn_model_path, spec_type='log', ml_algorithm='SVC'):
    if not os.path.exists("./spectrogram.png"):
        print("Please first run Create Spectrogram section.")
        return
    if not os.path.exists(cnn_model_path):
        print("Model path does not exists.")
        return


    random_state = 100
    cnn = load_model(cnn_model_path)

    spec = cv2.imread("./spectrogram.png")
    spec = cv2.resize(spec, (128, 128))
    spec = np.reshape(spec, [1, 128, 128, 3])


    classes = cnn.predict(spec)
    
    return emotions_dict[list(classes[0]).index(max(list(classes[0])))]   

# Config

In [72]:
# You can play with these.
spec_type = 'log'
order = 0
cnn_model_path = "/content/drive/MyDrive/colab/final_project/models/seed10/log_specAugment.h5"

# Not recommended to change these.
FRAME_SIZE = 2048
HOP_LENGTH = 512

# Run

In [118]:
audio_file = './ses_deneme_bir_ki.mp3'
ipd.Audio(audio_file)

1) Create Spectrogram (do not change parameters from here)

In [119]:
create_spectrogram(order=order, spec_type=spec_type) # ignore warning

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


2) Predict (do not change parameters from here)

In [120]:
predict_emotion(cnn_model_path=cnn_model_path, spec_type=spec_type)

'Fearful'